In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy
import ipywidgets
from ipywidgets import Output, Button, widgets, Layout
from IPython.display import HTML, clear_output, display
from base64 import b64encode
import requests
import io
import mysql.connector

import cryspy
from cryspy.B_parent_classes.cl_4_global import get_table_html_for_variables
from cryspy.H_functions_global.powder_experiments import report_powder_experiments
from cryspy.A_functions_base.database import DATABASE
from cryspy.A_functions_base.charge_form_factor import get_atom_name_ion_charge_shell, get_shell_orbit_electrons, calc_jl_for_shell_of_ion
from cryspy.A_functions_base.magnetic_form_factor import calc_j0, calc_j2
from cryspy.A_functions_base.function_2_space_group import ACCESIBLE_NAME_HM_EXTENDED, ACCESIBLE_NAME_HM_FULL, ACCESIBLE_NAME_HM_SHORT

In [2]:
COD_DB = mysql.connector.connect(
  host="sql.crystallography.net",
  user="cod_reader",
  password="",
  database="cod"
)
COD_CURSOR = COD_DB.cursor()

L_PARAMETERS = ["file", "doi", "year", "formula", "a", "b", "c", "alpha", "beta", "gamma", "sg", "title"]

In [3]:

def search_by_formula(request: str):
    request = request.strip()
    s_query_formula = "SELECT "+ ", ".join(L_PARAMETERS) + ' FROM data WHERE formula LIKE "%- ' + request + ' -%" ORDER BY year DESC'
    COD_CURSOR.execute(s_query_formula)
    result = COD_CURSOR.fetchall()
    if len(result) == 0:
        s_query_title = "SELECT "+ ", ".join(L_PARAMETERS) + ' FROM data WHERE title LIKE "%' + request + '%" ORDER BY year DESC'
        COD_CURSOR.execute(s_query_title)
        result = COD_CURSOR.fetchall()
    l_d_info = []
    for found_tuple in result:
        d_info = {}
        for name, val in zip(L_PARAMETERS, found_tuple):
            d_info[name] = val
        l_d_info.append(d_info)
    return l_d_info


In [4]:
def get_html_table(l_d_info, l_parameter_name):
    ls_html = ["<table><tr><th>index</th>"]
    for name in l_parameter_name:
        ls_html.append(f"<th>{name:}</th>")
    ls_html.append("</tr>")
    n_d_info = len(l_d_info)
    for i_d_info, d_info in enumerate(l_d_info):
        if i_d_info < 10:
            d_info_keys = d_info.keys()
            flag_doi = "doi" in d_info_keys
            if flag_doi:
                flag_doi = d_info["doi"] is not None
                if flag_doi:
                    url_doi = r"https://doi.org/" + d_info["doi"]
            ls_html.append("<tr>")
            ls_html.append(f"<td>{i_d_info+1:}</td>")
            for name in l_parameter_name:
                if name in d_info_keys:
                    if flag_doi and name.startswith("title"):
                        ls_html.append(f"<td><a href='{url_doi:}'>{d_info[name]:}</a></td>")
                    else:
                        ls_html.append(f"<td>{d_info[name]:}</td>")
                else:
                    ls_html.append("<td></td>")
            ls_html.append("</tr>")
        else:
            ls_html.append("<tr>")
            ls_html.append(f"<td>...</td>")
            ls_html.append("</tr>")
            break
    ls_html.append("</table>")
    return "\n".join(ls_html)

In [5]:
# global parameters
DDATA = {
    "rcif_obj": cryspy.str_to_globaln("global_"), 
    "file_rcif_name": "out.rcif",
    "SEARCH": [],
}
L_EXAMPLE = [
    ("PND: Single crystal", "rhochi_single_susceptibility_Ho2Ti2O7.rcif", "1xG6Dz7f78XinH78uPNu0Mo6eU8w5Lh0L"),
    ("PND: Powder 1D", "rhochi_polarized_powder_1d_Ho2Ti2O7_5K_1T.rcif", "18qdPs558CN39ABkR3JqtfNgvpn4SgCQo"),
    ("PND: Powder 2D", "rhochi_polarized_powder_2d_DyAl_5K_5T.rcif", "1GKnDsdKbYW6vhXuNQaS_jn1SddB6Viul"),
    ("Unpolarized neutron diffraction", "rhochi_unpolarized_powder_1d_PbSO4.rcif", "1wax6x5wnoFiQXArTWbWv-E3YTWUKF6EV"),
    ("X-ray diffraction", "rhochi_unpolarized_powder_1d_xrays_PbSO4.rcif", "1eQZLb613h7c0J5QA8fLlqhUCW6tI_-06"),
    ("TOF neutron diffraction", "rhochi_unpolarized_tof_powder_CeCuAl.rcif", "1qflJs3uXdG-O35rTGyyTXEqRa9WKELmX"),
    ("Spin density (MEM)", "mempy_spin_density_YTiO3.rcif", "1S8YPVcNyYKEkP8j4RkUB7X9vn85bm435"),
    ("Magnetization density (MEM)", "mempy_magnetization_density_Yb2Ti2O7_2K_1T.rcif", "1PQmBK0xDu7Zxs-QzmFAzX0HtJnGMQkdY"),
    ]


In [6]:
D_ELEMENTS = DATABASE["Elements"]
D_MFF = DATABASE["Magnetic Form Factor, tabulated"]
D_ISOTOPES = DATABASE["Isotopes"]
D_SA = DATABASE["Scattering amplitude"]
D_SO = DATABASE["Orbitals by radial Slater functions"]
D_DISPERSION = DATABASE["Dispersion"]
ACCESIBLE_NAME_HM = frozenset.union(frozenset(ACCESIBLE_NAME_HM_SHORT), ACCESIBLE_NAME_HM_FULL, ACCESIBLE_NAME_HM_EXTENDED)

In [7]:
L_ELEMENTS = [hh[1] for hh in D_ELEMENTS.keys() if hh[0].startswith("Atomic number")]

In [8]:
def get_info_about_element(type_symbol:str, shells: list):
    element, ion_charge, full_shell = get_atom_name_ion_charge_shell(type_symbol)
    l_key = ["Atomic number", "Classification", "Full_name", "Orbital occupancy", "Atomic weight", "b_scat_Re", "b_scat_Im"]
    sthovl = numpy.linspace(0,2,101)
    d_info = {"sthovl": sthovl}
    l_key_element = D_ELEMENTS.keys()
    for name in l_key:
        key_element = (name, element)
        if key_element in l_key_element:
            d_info[name]=D_ELEMENTS[key_element]

    kappa = 1.
    
    
    if len(shells) > 0:
        full_valence_shell = " ".join([f"{hh:}1" for hh in shells])
        try:
            mff_jl_rad = calc_jl_for_shell_of_ion(sthovl, type_symbol, full_valence_shell)
            d_info["mff_jl_rad"] = mff_jl_rad
        except UserWarning:
            pass

    l_key_mff = D_MFF.keys()
    key_mff = ("j0_AaBbCcD", element, ion_charge)
    if key_mff in l_key_mff:
        j0_parameters = D_MFF[key_mff]
        mff_j0_tab = calc_j0(sthovl, kappa, j0_parameters)[0]
        d_info["mff_j0_tab"] = mff_j0_tab
    key_mff = ("j2_AaBbCcD", element, ion_charge)
    if key_mff in l_key_mff:
        j2_parameters = D_MFF[key_mff]
        mff_j2_tab = calc_j2(sthovl, kappa, j2_parameters)[0]
        d_info["mff_j2_tab"] = mff_j2_tab
    return d_info

In [9]:
#  background="logo.png"
display(HTML("""
<hr>
<form width: 100%; align="center">
<a href="https://sites.google.com/view/cryspy/" title="CrysPy site"><img src="logo.png" alt="CrysPy logo" width="400"></a>
</form>
<hr>
<p align="center"><b>Web application</b> of the crystallographic python library CrysPy.
The library can be used as a handbook (click on the <i>"Handbook"</i> tab to start).
Upload your ".rcif" file or begin with one of the examples by selecting <i>"Upload File"</i> to start analyzing the diffraction data.
</p>
<hr>
"""))

In [10]:
# Reference: https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url

def download_file_from_google_drive(id):
    destination = io.BytesIO()
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)
    s_content = destination.getvalue().decode("ASCII")
    return s_content 

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    for chunk in response.iter_content(CHUNK_SIZE):
        if chunk: # filter out keep-alive new chunks
            destination.write(chunk)

In [11]:
def get_rcif_obj(number: int):
    if ((number >= 0) and (number < len(DDATA["SEARCH"]))):
        code_name = DDATA["SEARCH"][number]["file"]
        URL = r"http://www.crystallography.net/cod/" + str(code_name) + ".cif"
        session = requests.Session()
        response = session.get(URL, stream = True)
        destination = io.BytesIO()
        save_response_content(response, destination)
        s_content = destination.getvalue().decode("ASCII")
    else:
        s_content = ""
    return s_content

In [12]:
# global widgets

W_OUT = widgets.Output()

In [13]:
LAYOUT_15 = Layout(
    display='flex',    
    justify_content='center',
    # border='solid 2px',
    align_items='stretch',
    width='15%',
)
LAYOUT_70 = Layout(
    display='flex',    
    justify_content='center',
    # border='solid 2px',
    align_items='stretch',
    width='70%',
)

LAYOUT_100 = Layout(
    display='flex',
    flex_flow='row',
    justify_content='center',
    # border='solid 2px',
    align_items='stretch',
    width='99%',
    # height="400px",
)


In [14]:
# page MENU
def draw_page_menu():
    widget_fo = widgets.FileUpload(
        description = "Upload RCIF",
        accept='.rcif',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False,  # True to accept multiple files upload else False
    )

    def change_output(ddata):
        s_cont = ddata["new"][0]["content"].tobytes().decode("ASCII")
        DDATA["rcif_obj"] = cryspy.str_to_globaln(s_cont)
        W_OUT.clear_output()
        with W_OUT:
            print(ddata["new"][0]["name"])
            DDATA["file_rcif_name"]=ddata["new"][0]["name"]
            print("File was uploaded")
        DISPLAY_NEW_OBJECT()
    
    widget_fo.observe(change_output, names="value")
    
    def trigger_download(text, filename, kind='text/json'):
        # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
        content_b64 = b64encode(text.encode()).decode()
        data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
        js_code = f"""
            var a = document.createElement('a');
            a.setAttribute('download', '{filename}');
            a.setAttribute('href', '{data_url}');
            a.click()
        """
        with W_OUT:
            clear_output()
            display(HTML(f'<script>{js_code}</script>'))


    
    btn_download = Button(description='Download to PC')
    
    def download_rcif_file(e=None):
        file_name = DDATA["file_rcif_name"]
        if DDATA["rcif_obj"] is not None:
            trigger_download(DDATA["rcif_obj"].to_cif(), file_name, kind='text/plain')
        else:
            with W_OUT:
                print("File is not specified")
        
    btn_download.on_click(download_rcif_file)



    items_page = [
        widget_fo, 
        widgets.Label("Upload RCIF file from your local machine on server to perform the refinement"),
        btn_download, 
        widgets.Label("Download RCIF file from the server when the data analysis is completed")]
    
    page_menu = widgets.GridBox(
        items_page, layout=widgets.Layout(grid_template_columns="150px auto"))


    # page_menu = widgets.Box(
    #     [widget_fo, btn_download],
    #     layout=Layout(justify_content="center"))
    return page_menu

In [15]:
# page object
def draw_page_object():
    nrows = 10
    select_data_item = widgets.Select(
        options=[],
        description='',
        rows=nrows,
        tooltip="Select a data block",
        disabled=False,
        tabbable=True,
        layout=LAYOUT_100,
        )
    select_loop_item = widgets.Select(
        options=[],
        description='',
        disabled=True,
        tooltip="Select an item of the selected  data block",
        rows=nrows,
        layout=LAYOUT_100,
        )

    cif_text = widgets.Textarea(
        value='',
        placeholder="""Presentation of Item in RCIF format""",
        description='',
        disabled=False, # True
        rows=nrows,
        layout=LAYOUT_100,
    )
    
    button_save_changes = widgets.Button(
        description="Save changes of selected item", 
        layout=Layout(justify_content="center", width="99%"))

    button_add_to_datas = widgets.Button(
        description="Add to data blocks", 
        layout=Layout(justify_content="center", width="99%"))
    
    button_add_to_items = widgets.Button(
        description="Add to items", 
        layout=Layout(justify_content="center", width="99%"))

    button_del_data = widgets.Button(
        description="Delete data block", # the selected
        layout=Layout(justify_content="center", width="99%"))
    
    button_del_item = widgets.Button(
        description="Delete item", # the selected 
        layout=Layout(justify_content="center", width="99%"))

    def func_loop(dinfo_data_item):
        data_index = select_data_item.index
        data_item = DDATA["rcif_obj"].items[data_index]
        W_OUT.clear_output()
        with W_OUT:
            plt.close()
            l_plots = data_item.plots()
            if len(l_plots)>0:
                plt.show()            
            display(HTML(data_item.report_html()))
            # display(HTML(data_item._repr_html_()))
        if isinstance(data_item, cryspy.DataN):
            l_loop_item_name = [item.get_name() for item in data_item]
            select_loop_item.disabled = False
            select_loop_item.options = l_loop_item_name
            select_loop_item.index = None
            cif_text.value = ""
            cif_text.disabled = False # True
        else:
            cif_text.value = data_item.to_cif()
            select_loop_item.options = ()
            select_loop_item.disabled = True
            cif_text.disabled = False
        return
    
    select_data_item.observe(func_loop, names="index")
    
    def show_cif(dinfo_loop_item):
        data_index = select_data_item.index
        data_item = DDATA["rcif_obj"].items[data_index]
        loop_index = select_loop_item.index
        if loop_index is None:
            cif_text.value = data_item.to_cif()
            selected_item = data_item
        else:
            loop_item = data_item.items[loop_index]
            cif_text.value = loop_item.to_cif()
            selected_item = loop_item
        cif_text.disabled = False
        W_OUT.clear_output()
        with W_OUT:
            plt.close()
            l_selected_item_plots = selected_item.plots()
            if len(l_selected_item_plots)>0:
                plt.show()
            display(HTML(selected_item.report_html()))
            # display(HTML(loop_item._repr_html_()))

        
    select_loop_item.observe(show_cif, names="index")
    
    def save_cif(b):        
        data_index = select_data_item.index
        data_item = DDATA["rcif_obj"].items[data_index]
        loop_index = select_loop_item.index
        
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            loop_item_new = globaln_new.items[0]
            if isinstance(loop_item_new, (cryspy.LoopN, cryspy.ItemN)):
                if loop_index is None:
                    DDATA["rcif_obj"].items.pop(data_index)
                    DDATA["rcif_obj"].items.insert(data_index, loop_item_new)
                else:
                    data_item.items.pop(loop_index)
                    data_item.items.insert(loop_index, loop_item_new)
                show_cif({})

    def add_to_globaln(b):        
        data_index = select_data_item.index
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            l_data_loop_item_new = globaln_new.items
            for data_loop_item_new in l_data_loop_item_new:
                if isinstance(data_loop_item_new, (cryspy.LoopN, cryspy.ItemN, cryspy.DataN)):
                    DDATA["rcif_obj"].items.append(data_loop_item_new)
            DISPLAY_NEW_OBJECT()

    def add_to_datan(b):        
        data_index = select_data_item.index
        l_data_item = DDATA["rcif_obj"].items[data_index]
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            l_loop_item_new = globaln_new.items
            for loop_item_new in l_loop_item_new:
                if isinstance(loop_item_new, (cryspy.LoopN, cryspy.ItemN)):
                    l_data_item.items.append(loop_item_new)
            DISPLAY_NEW_OBJECT()
    
    def del_data(b):        
        data_index = select_data_item.index
        if data_index is not None:
            DDATA["rcif_obj"].items.pop(data_index)
            DISPLAY_NEW_OBJECT()

    def del_item(b):        
        data_index = select_data_item.index
        if data_index is not None:
            l_data_item = DDATA["rcif_obj"].items[data_index]
            loop_index = select_loop_item.index
            if loop_index is not None:
                l_data_item.items.pop(loop_index)
                DISPLAY_NEW_OBJECT()


    button_save_changes.on_click(save_cif)
    button_add_to_datas.on_click(add_to_globaln)
    button_add_to_items.on_click(add_to_datan)
    button_del_data.on_click(del_data)
    button_del_item.on_click(del_item)
    
    items = [
        widgets.Label("Data blocks:"), #  (click to select)
        widgets.Label("Items of data block:"), # selected 
        widgets.Label("Selected Item:"), 
        select_data_item, select_loop_item,cif_text, 
        button_del_data, button_del_item, button_save_changes,
        button_add_to_datas,button_add_to_items,
        ]
    # page_object = widgets.HBox(
    #     items, 
    #     layout=Layout(flex_flow="row wrap", justify_content='center'))
        
    page_object =widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 150px) auto"))

    return page_object, select_data_item, select_loop_item, cif_text

In [16]:
def draw_page_cryspy_methods():
    button_info = widgets.Button(description="Info")
    button_calculations = widgets.Button(description="Compute")
    button_fitting = widgets.Button(description="Refinement")
    button_sim_pd = widgets.Button(description="Simulation PD")
    button_mempy_sd = widgets.Button(description="Spin density (MEM)")
    button_mempy_md = widgets.Button(description="Magn. density (MEM)")

    items = [button_info, button_calculations, button_fitting, button_sim_pd, button_mempy_sd, button_mempy_md]
    
    def disabled_buttons(flag:bool=False):
        for button in items:
            button.disabled = flag

    def print_object_info(b):
        disabled_buttons(True)
        rcif_object = DDATA["rcif_obj"]
        
        ls_html = []
        ls_html.append(get_table_html_for_variables(rcif_object))
        ls_html.append(rcif_object.report_html())
        s_powder = report_powder_experiments(DDATA["rcif_obj"])
        W_OUT.clear_output()
        with W_OUT:
            display(HTML("\n".join(ls_html)))
            print(s_powder)
        disabled_buttons(False)
        return 

    def run_fitting(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.rhochi_rietveld_refinement(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()            
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    def run_simulation_pd(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Simulation is running...")
            l_crystal = [item for item in DDATA["rcif_obj"] if isinstance(item, cryspy.Crystal)]
            pd = cryspy.H_functions_global.simulations.simulation_pd(l_crystal)
            DDATA["rcif_obj"].add_items([pd, ])
            print("Simulation completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()            
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    def run_chi_sq(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.rhochi_no_refinement(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    def run_mempy_sd(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.mempy_spin_density_reconstruction(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    def run_mempy_md(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.mempy_magnetization_density_reconstruction(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    button_info.on_click(print_object_info)
    button_fitting.on_click(run_fitting)
    button_sim_pd.on_click(run_simulation_pd)
    button_calculations.on_click(run_chi_sq)
    button_mempy_sd.on_click(run_mempy_sd)
    button_mempy_md.on_click(run_mempy_md)

    items_page = [
        button_info, 
        widgets.Label("Get general information about RCIF object like number of variables"),
        button_calculations, 
        widgets.Label("Perform calculations of experimental data without refinement"), 
        button_fitting, 
        widgets.Label("Refinement of parameters to minimize GoF"), 
        button_sim_pd,
        widgets.Label("Simulation of neutron powder diffraction for defined crystal structures"), 
        button_mempy_sd, 
        widgets.Label("Spin density reconstruction by maximum entropy procedure"), 
        button_mempy_md, 
        widgets.Label("Magnetization density reconstruction by maximum entropy procedure")]
    
    page_cryspy_methods = widgets.GridBox(
        items_page, layout=widgets.Layout(grid_template_columns="150px auto"))


    return page_cryspy_methods

In [17]:
def draw_page_examples():
    btn_1 = Button(description=L_EXAMPLE[0][0], tooltip="Ho2Ti2O7")
    btn_2 = Button(description=L_EXAMPLE[1][0], tooltip="Ho2Ti2O7")
    btn_3 = Button(description=L_EXAMPLE[2][0], tooltip="DyAl garnet")
    btn_4 = Button(description=L_EXAMPLE[3][0], tooltip="PbSO4")
    btn_5 = Button(description=L_EXAMPLE[4][0], tooltip="PbSO4")
    btn_6 = Button(description=L_EXAMPLE[5][0], tooltip="CeCuAl")
    btn_7 = Button(description=L_EXAMPLE[6][0], tooltip="YTiO3")
    btn_8 = Button(description=L_EXAMPLE[7][0], tooltip="Yb2Ti2O7")

    def download_example(b):
        l_name = [hh[0] for hh in L_EXAMPLE]
        ind_example = l_name.index(b.description)
        file_id = L_EXAMPLE[ind_example][2]
        s_content = download_file_from_google_drive(file_id)
        
        DDATA["rcif_obj"]=cryspy.str_to_globaln(s_content)
        DDATA["file_rcif_name"] = L_EXAMPLE[ind_example][1]
        DISPLAY_NEW_OBJECT()
        return 

    btn_1.on_click(download_example)
    btn_2.on_click(download_example)
    btn_3.on_click(download_example)
    btn_4.on_click(download_example)
    btn_5.on_click(download_example)
    btn_6.on_click(download_example)
    btn_7.on_click(download_example)
    btn_8.on_click(download_example)


    items_page = [
        btn_1, 
        widgets.Label("Susceptibility tensor from single crystal diffraction data analysis of Ho2Ti2O7 on 6T2@LLB at 5K 1T"),
        btn_2, 
        widgets.Label("Susceptibility tensor from powder diffraction data analysis of Ho2Ti2O7 on 6T2@LLB at 5K 1T"), 
        btn_3, 
        widgets.Label("Magnetically induced preferred orientation in dysprosium aluminum garnet measured on 6T2@LLB at 5K 5T"), 
        btn_4, 
        widgets.Label("Refinement of diffraction pattern for PbSO4 taken from FullProf examples "), 
        btn_5, 
        widgets.Label("Refinement of X-ray powder diffraction data for PbSO4"),
        btn_6, 
        widgets.Label("Refinement of TOF neutron diffraction data for CeCuAl"), 
        btn_7, 
        widgets.Label("Reconstruction of spin density in YTiO3 based on flipping ratio measurements on the diffractometer 6T2@LLB"), 
        btn_8, 
        widgets.Label("Reconstruction of magnetization density in the presence of local anisotropy in Yb2Ti2O7. Flipping ratio measurements on single-crystal performed on 6T2@LLB at 2K 1T")]
    
    w_examples = widgets.GridBox(
        items_page, layout=widgets.Layout(grid_template_columns="150px auto"))
    return w_examples

In [18]:
# page handbook
def draw_page_handbook():
    
    tags = widgets.TagsInput(
        value=[],
        placeholder="Search Form",
        tooltip="Examples: 'O', 'O2-', 'O2- 2p', 'F d -3 m', 'Fe2 O3' 'aluminum garnet'.",
        
        allow_duplicates=False
    )
    
    def get_info_about_elements(b):
        ls_html = ["<table><tr><th>Chemical symbol</th>"]
        l_d_info = []
        l_sg = []
        flag_mff = False
        flag_element = False
        for input_string in tags.value:
            flag = True
            l_input_param = input_string.strip().split()
            type_symbol = l_input_param[0].capitalize()
            shells = l_input_param[1:]
            flag_sg = (input_string in ACCESIBLE_NAME_HM)
            if flag_sg:
                l_sg.append(cryspy.SpaceGroup(name_hm_alt=input_string))
                flag=False
            else:
                try:
                    element, ion_charge, full_shell = get_atom_name_ion_charge_shell(type_symbol)
                except KeyError:
                    flag=False
            if flag:
                flag_element = True
                d_info = get_info_about_element(type_symbol, shells=shells)
                d_info["Type symbol"] = type_symbol
                flag_mff = (flag_mff | (
                    ("mff_jl_rad" in d_info.keys()) | ("mff_j0_tab" in d_info.keys())
                    ))
                d_info["Element"] = element
                d_info["Shell"] = " ".join(shells)
                d_info["Oxygen state"] = ion_charge
                d_info["Electron configuration"] = d_info["Orbital occupancy"]
                if ion_charge != 0:
                    if len(full_shell.strip().split()) > 3:
                        full_shell = "... "+" ".join(full_shell.strip().split()[-3:])
                    d_info["Electron configuration"] = full_shell
                l_d_info.append(d_info)
                ls_html.append(f"<th>{d_info['Element']:}</th>")
        ls_html.append("</tr>")
        l_table = ["Full_name", "Classification", "Atomic number", "Oxygen state", "Electron configuration", "Atomic weight", "b_scat_Re", "b_scat_Im"]
        for name_table in l_table:
            ls_html.append(f"<tr><th>{name_table:}</th>")
            for d_info in l_d_info:
                if name_table in d_info.keys():
                    ls_html.append(f"<td>{d_info[name_table]:}</td>")
                else:
                    ls_html.append(f"<td></td>")
            ls_html.append("</tr>")
        ls_html.append("</table>")
        
        W_OUT.clear_output()
        plt.close()
        with W_OUT:
            if flag_element:
                display(HTML("\n".join(ls_html)))
            if flag_mff:
                fig, ax = plt.subplots()
                ax.set_xlabel(r"sin $\theta$ / $\lambda$ ($\AA^{-1}$)")
                ax.set_ylabel("magnetic form-factor")
                for d_info in l_d_info:
                    
                    if ("mff_jl_rad" in d_info.keys()):
                        sthovl = d_info["sthovl"]
                        mff_jl_rad = d_info["mff_jl_rad"]
                        ax.plot(sthovl, mff_jl_rad[:, 0], "-", label=f"<j0> {d_info['Type symbol']:} {d_info['Shell']:}", alpha=0.7)
                        ax.plot(sthovl, mff_jl_rad[:, 2], ":", label=f"<j2> {d_info['Type symbol']:} {d_info['Shell']:}", alpha=0.7)
                    if ("mff_j0_tab" in d_info.keys()):
                        ax.plot(d_info["sthovl"], d_info["mff_j0_tab"], "-", label=f"<j0> {d_info['Type symbol']:} tab.", alpha=0.7)
                    if ("mff_j2_tab" in d_info.keys()):
                        ax.plot(d_info["sthovl"], d_info["mff_j2_tab"], ":", label=f"<j2> {d_info['Type symbol']:} tab.", alpha=0.7)
                ax.set_xlim((0., 2.))
                ax.legend()
                plt.show()
            if len(l_sg) > 0:
                for sg in l_sg:
                    display(HTML(sg.report_html()))
        tags.value = []

    def get_info_about_crystal_structure(b):
        if len(tags.value) > 0:
            request = tags.value[0]
            DDATA["SEARCH"] = search_by_formula(request)
            
            W_OUT.clear_output()
            with W_OUT:
                s_table = get_html_table(DDATA["SEARCH"], L_PARAMETERS[2:])
                display(HTML(s_table))
        else:
            W_OUT.clear_output()
            with W_OUT:
                print("Type info about searched structure like 'Fe2 O3' or 'water'")
        tags.value = []

    def add_to_rcif_file(b):
        W_OUT.clear_output()
        if len(tags.value) > 0:
            request = tags.value[0]
        else:
            with W_OUT:
                print("Type number of element to add in RCIF file like '1' or '2'")
            return
        try:
            request = int(request)
        except:
            with W_OUT:
                print("Request should be an integer")
            return
            
        flag = ((request > 0) and (request <= len(DDATA["SEARCH"])))
        if not flag:
            with W_OUT:
                print(f"Request should be positive and no more than {len(DDATA['SEARCH']):}")
            return

        s_content = get_rcif_obj(request-1)
        try:
            obj_rcif = cryspy.str_to_globaln(s_content)
            DDATA["rcif_obj"].add_items(obj_rcif.items)
            DISPLAY_NEW_OBJECT()
        except:
            with W_OUT:
                print("It caused the problem with converting the CIF to RCIF objects.")
                print(s_content)
            
        tags.value = []

    button = Button(description="Chemical elements", tooltip="Type in the search form: 'O', 'Fe3+ 3d'; and press this button")
    button.on_click(get_info_about_elements)

    button_cod = Button(description="Crystal structures", tooltip="Type in the search form: 'Fe3 O4', 'aluminum garnet'; and press this button")
    button_cod.on_click(get_info_about_crystal_structure)

    button_add_cif = Button(description="Add to RCIF file", tooltip="Type in the search form the index of found structure: '1', '2'; and press this button")
    button_add_cif.on_click(add_to_rcif_file)

    page_handbook = widgets.GridBox(
        [widgets.Label(""), tags,
        button, widgets.Label("Get information about chemical elements and their magnetic form-factors"), 
        button_cod, widgets.Label("Search the crystal structures by their formula or name. Powered by COD: http://www.crystallography.net/cod/"), 
        button_add_cif, widgets.Label("Add the found crystal structure to your RCIF file by typing its index")
        ], layout=widgets.Layout(grid_template_columns="150px auto"))

    return page_handbook


In [19]:
page_menu = draw_page_menu()
page_examples = draw_page_examples()
page_object, SELECT_DATA_ITEM, SELECT_LOOP_ITEM, CIF_TEXT = draw_page_object()
page_cryspy_methods = draw_page_cryspy_methods()
page_handbook = draw_page_handbook()

In [20]:
ACCORDION = widgets.Accordion(
            children=[page_handbook, 
            widgets.VBox([page_menu, widgets.Label("Examples:"), page_examples]), 
            page_object, ],
            titles=('Handbook', "Upload File", "RCIF File")
            )

In [21]:
accordion_methods = widgets.Accordion(
    children=[page_cryspy_methods,],
    titles = ("RhoChi and MEMPy procedures", )
    )

In [23]:
display(ACCORDION, accordion_methods, W_OUT)

Accordion(children=(GridBox(children=(Button(description='Get Info', style=ButtonStyle()), TagsInput(value=[],…

Accordion(children=(GridBox(children=(Button(description='Info', style=ButtonStyle()), Label(value='Get genera…

Output()

In [24]:
def DISPLAY_NEW_OBJECT():
    l_data_item_name = [item.get_name() for item in DDATA["rcif_obj"].items]
    
    SELECT_DATA_ITEM.options = l_data_item_name
    # if (SELECT_DATA_ITEM.index is None) and len(l_data_item_name)>0:
    #     SELECT_DATA_ITEM.index = 0
    ACCORDION.selected_index = 2
    if (SELECT_DATA_ITEM.index is None):
        return
    
    data_index = SELECT_DATA_ITEM.index
    if data_index >= len(l_data_item_name):
        data_index = 0
         
    data_item = DDATA["rcif_obj"].items[data_index]
    if isinstance(data_item, cryspy.DataN):
        l_loop_item_name = [item.get_name() for item in data_item.items]
        SELECT_LOOP_ITEM.options = l_loop_item_name

        # if (SELECT_LOOP_ITEM.index is None) and len(l_loop_item_name)>0:
        #     SELECT_LOOP_ITEM.index = 0
        if (SELECT_LOOP_ITEM.index is None):
            return
        loop_index = SELECT_LOOP_ITEM.index
        if loop_index >= len(l_loop_item_name):
            loop_index = 0
        loop_item = data_item.items[0]
        CIF_TEXT.value = loop_item.to_cif()
    else:
        CIF_TEXT.value = data_item.to_cif()
        